# Importing Libraries
***

In [ ]:
from neo4j import GraphDatabase
from pandas import DataFrame
import pandas as pd
import numpy as np
import re
import csv
import ast
import json

# Connecting to Neo4j
***

In [ ]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "123123123"))

# Deffining Useful Functions
***

In [ ]:
def run_query(query): 
     with driver.session() as session:
        result = session.run(query)
        #print(result.single()[0])
        
        
def delete_all_nodes():
    query = '''
    MATCH(n) DETACH DELETE(n)
    '''
    run_query(query)

def record_count():
    with driver.session() as session:
        result = session.run("MATCH (n) RETURN count(n) LIMIT 100")
        print(result.single())

def delete_nodes(var):
    query = f'''
    MATCH (c:{var})
    DELETE c

    '''
    run_query(query)

# Adding Organization (Affiliation) Data to Author Nodes
***

In [ ]:
df_auth = pd.read_csv("./data/authors.csv", encoding='utf-8')

In [ ]:
print(len(df_auth['org'].unique()))
df_auth.shape

14734


(27765, 3)

In [ ]:
query = """
    LOAD CSV WITH HEADERS FROM "file:///C:/Users/odyky/Desktop/Property_Graph_Research_Publications/data/authors.csv" AS x
    MATCH (a:Author {id: x.id})
    SET a.org = x.org
    """
run_query(query=query)

# Reading Reviewers Data
***

In [ ]:
df = pd.read_csv("./data/papers_reviewers.csv", encoding='utf-8')
df.head(5)

,PaperID,ReviewerID
0,53e9b41bb7602d9703f14db9,53f468dcdabfaeb22f54794f
1,53e9b41bb7602d9703f14db9,53f39bdadabfae4b34a98b5e
2,53e9b41bb7602d9703f14db9,53f42b9adabfaedd74d1b267
3,53e9b40eb7602d9703f04298,53f784eedabfae92b40c203a
4,53e9b40eb7602d9703f04298,562ca9cf45cedb3398c8b9f5


## Generate Artificial Response Values

In [ ]:
x = int(len(df) / 3)  # number of times to repeat the pattern
result = [] # create an empty list to store the result
description = [] # create an empty list to store the result

for i in range(x):
    result.append("yes")
    result.append("yes")
    result.append("no")
    description.append("Excellent Work. It can be Published")
    description.append("Excellent Work. It can be Published")
    description.append("Have minor mistakes. Need modifications in the following parts:...")

df['response'] = result
df['description'] = description

In [ ]:
df.head()

,PaperID,ReviewerID,response,description
0,53e9b41bb7602d9703f14db9,53f468dcdabfaeb22f54794f,yes,Excellent Work. It can be Published
1,53e9b41bb7602d9703f14db9,53f39bdadabfae4b34a98b5e,yes,Excellent Work. It can be Published
2,53e9b41bb7602d9703f14db9,53f42b9adabfaedd74d1b267,no,Have minor mistakes. Need modifications in the...
3,53e9b40eb7602d9703f04298,53f784eedabfae92b40c203a,yes,Excellent Work. It can be Published
4,53e9b40eb7602d9703f04298,562ca9cf45cedb3398c8b9f5,yes,Excellent Work. It can be Published


In [ ]:
df.to_csv("./data/papers_reviewers.csv", encoding='utf-8', index=False)

# Adding Response and Description Data to Reviews Relationship
***

In [ ]:
query = """
    LOAD CSV WITH HEADERS FROM "file:///C:/Users/odyky/Desktop/Property_Graph_Research_Publications/data/papers_reviewers.csv" AS x
    MATCH (a:Author {id: x.ReviewerID}) - [r:reviews] -> (c:Paper {id: x.PaperID})
    SET r.decision = x.response
    SET r.description = x.description
    """
run_query(query=query)